In [ ]:
import csv
print(csv.__file__)

import os
import shutil
import pandas as pd
import numpy as np
from pandas import read_csv
from numpy import genfromtxt
from io import StringIO


# RNA

In [ ]:
def downloadRnaData(fname, **kwargs):
    def rewrite_csv_as_tab(fname):
        with open(fname, newline='') as fp:
            dialect = csv.Sniffer().sniff(fp.readline())

            fp.seek(0)
            for row in csv.reader(fp, dialect):
                yield "\t".join(row)
    return np.recfromcsv(rewrite_csv_as_tab(fname), delimiter='\t', encoding=None, **kwargs, skip_header=5)

def downloadClinicalData(fname, **kwargs):
    def rewrite_csv_as_tab(fname):
        with open(fname, newline='') as fp:
            dialect = csv.Sniffer().sniff(fp.readline())
            fp.seek(0)
            for row in csv.reader(fp, dialect):
                yield "\t".join(row)
    return np.recfromcsv(rewrite_csv_as_tab(fname), delimiter='\t', encoding=None, **kwargs)

def testOpenFiles(directory):
    for root,dirs,files in os.walk(directory):
        files = [f for f in files if not f[0] == '.']
        dirs[:] = [d for d in dirs if not d[0] == '.']
        for file in files:
            path = directory + '/' + file
            print(path)
            #patient.append(file[0:file.find('.')])
            
            #openFile = pd.read_csv(path,sep='\t', header=5)
            downloadRnaData(path, case_sensitive=True)
            downloadClinicalData(path, case_sensitive=True)
            
            
def getPatientVector(directory, csv):
    data=downloadRnaData(csv)
    features=[]
    features.append('patient_id')
    
    #for root,dirs,files in os.walk(directory):
        #for file in files:
            #if file.endswith(".tsv"):
                #path = directory + '/' + file
                #features.append(file)
    for row in data:
        features.append(row[1])
    return(features)

def getCaseID(samplesheet, filename):
    caseID = 0
    samplesheet = downloadClinicalData(samplesheet)
    for row in samplesheet:
        #print(row.iloc[1])
        if row[1] == filename:
            caseID = row[5]
    return caseID


def findPatient(clinicalFile, patient):
    clinicalData = downloadClinicalData(clinicalFile)
    for row in clinicalData:
        if (row[1] == patient):
            return row[27]
    return None
            
def writeToCsv(outfile, infile):
    List_columns = infile[0]
    infile = infile[1:]
    df = pd.DataFrame(infile)
    df.to_csv(outfile, index=False, header=List_columns)

def fillMatrix(directory, csv, samplesheet):
    features = getPatientVector(directory, csv) 
    matrix = []
    matrix.append(features)
    #row = []

    
    for root,dirs,files in os.walk(directory):
        files = [f for f in files if not f[0] == '.']
        dirs[:] = [d for d in dirs if not d[0] == '.']
        for file in files:
            if file.endswith(".tsv"):
                print(file)
                row = []
                path = directory + '/' + file
                openFile = downloadRnaData(path, case_sensitive=True)
            
                patientID = getCaseID(samplesheet, file)
                row.append(patientID)
                for x in openFile:
                    row.append(x[7])
                
                matrix.append(row)
            #row.clear()
            
        #print(matrix)
        #os.makedirs('/Users/soniasharapova/Documents/JunDingResearch/MultiOmics/rnaSeq', exist_ok=True)  
        #matrix.to_csv('/Users/soniasharapova/Documents/JunDingResearch/MultiOmics/rnaSeq/outTest.csv')
        
        #matrix.to_csv('/Users/soniasharapova/Documents/JunDingResearch/MultiOmics/rnaSeq/outTest.csv', index = False)

        
        
    
        print(matrix)
        return(matrix)
    
    
def addYforPatient(genePatientMatrix, clinicalFile, outfile):

    for row in genePatientMatrix:
        patientID = row[0]
        findPat = findPatient(clinicalFile, patientID)
        row.append(findPat)
        
    writeToCsv(outfile, genePatientMatrix)
    return genePatientMatrix
    
    
    
    
    

In [ ]:
testrnaFolder = "/Users/soniasharapova/Documents/JunDingResearch/MultiOmics/rnaSeq/testSet/newTest/0f382cf1-3e52-4078-bf12-cc6ba998ed3f.rna_seq.augmented_star_gene_counts.tsv"
rnaFolder = '/Users/soniasharapova/Documents/JunDingResearch/MultiOmics/rnaSeq/testSet/newTest'
clinical = '/Users/soniasharapova/Documents/JunDingResearch/MultiOmics/clinical.project-TCGA-BRCA.2022-07-26/clinical.tsv'
sample_sheet = '/Users/soniasharapova/Downloads/rnaSeq_sample_sheet.tsv'

directoryRna = os.path.join("c:\\",rnaFolder)
#getPatientVector(directoryRna, testrnaFolder)

#getCaseID(sample_sheet, "0f382cf1-3e52-4078-bf12-cc6ba998ed3f.rna_seq.augmented_star_gene_counts.tsv")

matrix = fillMatrix(directoryRna, testrnaFolder, sample_sheet)
print(matrix[-1])


In [ ]:
fullMatrix = addYforPatient(matrix, clinical)
print(fullMatrix[-1])

In [ ]:
os.getcwd()
os.chdir("../../..")
os.getcwd()
os.listdir()
os.chdir("/Volumes/Toshiba_blue")
os.listdir()
os.getcwd()

In [ ]:
os.getcwd()
rnaFolder = './rnaSeq/rnaSeqFiles/files'
clinical = './clinical.tsv'
sample_sheet = './rnaSeq/rnaSeqSampleSheet.tsv'
#directoryRna = os.path.join("c:\\",rnaFolder)
testrnaFolder = './rnaSeq/rnaSeqFiles/files/0a9e33db-2527-4cc3-8669-a7c10fed7a7f.rna_seq.augmented_star_gene_counts.tsv'

#testrnaFolder = "/Users/soniasharapova/Documents/JunDingResearch/MultiOmics/rnaSeq/testSet/newTest/0f382cf1-3e52-4078-bf12-cc6ba998ed3f.rna_seq.augmented_star_gene_counts.tsv"

#test =          '/Users/soniasharapova/Documents/JunDingResearch/MultiOmics/rnaSeq/testSet/newTest/1a3dcc95-64bf-4886-9807-9511b24f30af.rna_seq.augmented_star_gene_counts.tsv'

#patients = getPatientID(directoryRna)
#sample_sheet = '/Users/soniasharapova/Downloads/rnaSeq_sample_sheet.tsv'
outfile = './rnaSeq/rnaSeqFiles/outFile.csv'
matrix = fillMatrix(rnaFolder, testrnaFolder, sample_sheet)
fullMatrix = addYforPatient(matrix, clinical, outfile)

print(fullMatrix)
#addYforPatient(matrix, clinical)

# miRna

In [ ]:
def downloadMiRnaData(fname, **kwargs):
    def rewrite_csv_as_tab(fname):
        with open(fname, newline='') as fp:
            dialect = csv.Sniffer().sniff(fp.readline())

            fp.seek(0)
            for row in csv.reader(fp, dialect):
                yield "\t".join(row)
    return np.recfromcsv(rewrite_csv_as_tab(fname), delimiter='\t', encoding=None, **kwargs)

def downloadClinicalData(fname, **kwargs):
    def rewrite_csv_as_tab(fname):
        with open(fname, newline='') as fp:
            dialect = csv.Sniffer().sniff(fp.readline())
            fp.seek(0)
            for row in csv.reader(fp, dialect):
                yield "\t".join(row)
    return np.recfromcsv(rewrite_csv_as_tab(fname), delimiter='\t', encoding=None, **kwargs)

def testOpenFiles(directory):
    for root,dirs,files in os.walk(directory):
        files = [f for f in files if not f[0] == '.']
        dirs[:] = [d for d in dirs if not d[0] == '.']
        for file in files:
            path = directory + '/' + file
            #print(path)
            #patient.append(file[0:file.find('.')])
            
            #openFile = pd.read_csv(path,sep='\t', header=5)
            downloadMiRnaData(path, case_sensitive=True)
            downloadClinicalData(path, case_sensitive=True)
            co
    
    
    
    
def getUniqueID(data):
    uniqueArray = []
    for row in data:
        ident = row[0]
        if ident not in uniqueArray:
            uniqueArray.append(ident)
    return uniqueArray
            
            
            
def getPatientVector(directory, csv):
    data=downloadMiRnaData(csv)
    features=[]
    features.append('patient_id')
    features.extend(getUniqueID(data))
    
    return(features)




def getCaseID(samplesheet, filename):
    caseID = 0
    samplesheet = downloadClinicalData(samplesheet)
    for row in samplesheet:
        #print(row.iloc[1])
        
        if row[1] == filename:
            #print(row[1])
            print(filename)
            caseID = row[5]
    return caseID



def miRnaToGeneName(samplesheet, miRnaID):
    caseID = 0
    samplesheet = downloadClinicalData(samplesheet)
    for row in samplesheet:
        if row[1] == miRnaID:
            caseID = row[1]
    return caseID


def findPatientY(clinicalFile, patient):
    clinicalData = downloadClinicalData(clinicalFile)
    for row in clinicalData:
        if (row[1] == patient):
            
            return row[27]
    print(row[1],patient)
    return None

def getAvgReadCount(unique, csvData):
    sumArray = []
    
    for ident in unique:
        summ = 0
        count = 1
        for row in csvData:
            if row[0] == ident:
                summ = row[2] + summ
                count = count + 1
        if count == 1:
            avg = int(summ / count)
        else:
            avg = int(summ / (count - 1) )
        sumArray.append(avg)
            
    return sumArray



def getReadCountMirna(csvData):
    readCount = []
    count = 0
    for row in csvData:
        count = row[1]
        readCount.append(count)
        count = 0
    return readCount
            
def writeToCsv(outfile, infile):
    List_columns = infile[0]
    infile = infile[1:]
    df = pd.DataFrame(infile)
    df.to_csv(outfile, index=False, header=List_columns)

def fillMatrix(directory, csv, samplesheet):
    features = getPatientVector(directory, csv) 
    matrix = []
    matrix.append(features)
    #row = []
    openCSV = downloadMiRnaData(csv, case_sensitive=True)
    unique = getUniqueID(openCSV)
    
    for root,dirs,files in os.walk(directory):
        files = [f for f in files if not (f[0] == '.' or f == 'annotations.txt' or not f.endswith(".txt"))]
        dirs[:] = [d for d in dirs if not d[0] == '.']
        for file in files:
            row = []
            path = directory + '/' + file
            openFile = downloadMiRnaData(path, case_sensitive=True)
            patientID = getCaseID(samplesheet, file)
            row.append(patientID)
            #row.extend(getAvgReadCount(unique, openFile))
            row.extend(getReadCountMirna(openFile))
            matrix.append(row)
    
        print(matrix)
        return(matrix)
    
    
def addYforPatient(genePatientMatrix, clinicalFile, outfile):
    line = 0
    for row in genePatientMatrix:
        line = line+1
        print(line)
        patientID = row[0]
        findPat = findPatientY(clinicalFile, patientID)
        print('findPat',row[0], findPat)
        row.append(findPat)
        
    writeToCsv(outfile, genePatientMatrix)
    return genePatientMatrix
    
    
    
    
    

In [ ]:
miRnaFolder = '/Users/soniasharapova/Documents/JunDingResearch/MultiOmics/miRNASeq/testSet/42files'
csvEx = '/Users/soniasharapova/Documents/JunDingResearch/MultiOmics/miRNASeq/testSet/42files/0bb63e02-f74a-4acc-b000-b87bd5abcd71.mirbase21.isoforms.quantification.txt'
clinical = '/Users/soniasharapova/Documents/JunDingResearch/MultiOmics/clinical.project-TCGA-BRCA.2022-07-26/clinical.tsv'
sample_sheet = '/Users/soniasharapova/Documents/JunDingResearch/MultiOmics/miRNASeq/miRna_sample_sheet.tsv'


directoryMiRna = os.path.join("c:\\",miRnaFolder)

x = fillMatrix(directoryMiRna, csvEx, sample_sheet)


type(x)

In [ ]:
os.getcwd()
os.chdir("../../..")
os.getcwd()
os.listdir()
os.chdir("/Volumes/Toshiba_blue")
os.listdir()
os.getcwd()

In [ ]:
os.getcwd()
miRnaFolder = './miRnaSeq/miRnaSeqFiles/mirnaFilesOnly'
clinical = './clinical.tsv'
sample_sheet = './miRnaSeq/miRnaSeqSampleSheet.tsv'
#directoryRna = os.path.join("c:\\",rnaFolder)
testMiRnaFolder = './miRnaSeq/miRnaSeqFiles/files/0a00ab98-e267-45f9-89af-5fecf427aca9.mirbase21.mirnas.quantification.txt'

#testrnaFolder = "/Users/soniasharapova/Documents/JunDingResearch/MultiOmics/rnaSeq/testSet/newTest/0f382cf1-3e52-4078-bf12-cc6ba998ed3f.rna_seq.augmented_star_gene_counts.tsv"

#test =          '/Users/soniasharapova/Documents/JunDingResearch/MultiOmics/rnaSeq/testSet/newTest/1a3dcc95-64bf-4886-9807-9511b24f30af.rna_seq.augmented_star_gene_counts.tsv'

#patients = getPatientID(directoryRna)
#sample_sheet = '/Users/soniasharapova/Downloads/rnaSeq_sample_sheet.tsv'
miRnaOutfile = './miRnaSeq/miRnaSeqFiles/outFile.csv'
miRnaMatrix = fillMatrix(miRnaFolder, testMiRnaFolder, sample_sheet)



30671b2b-1583-4908-bfab-25a9ccb16997.mirbase21.mirnas.quantification.txt
3068d8b1-c407-4ee2-a7e2-582138927142.mirbase21.mirnas.quantification.txt
3074c972-81b2-44b6-8684-fb421c304389.mirbase21.mirnas.quantification.txt
30d5997d-396b-4345-b490-4b0879b975ef.mirbase21.mirnas.quantification.txt
31268895-fd75-451e-8044-e1223555b79c.mirbase21.mirnas.quantification.txt
3163c420-eccd-4bf2-9bb6-170f3549556c.mirbase21.mirnas.quantification.txt
316b1936-e040-4ea3-8346-d00b2b98cf0a.mirbase21.mirnas.quantification.txt
31ce1d57-ce67-4c6f-bd4c-0fbcbf9cdbe5.mirbase21.mirnas.quantification.txt
31ce585a-95d2-4b0b-b9cd-d4f86f994ef0.mirbase21.mirnas.quantification.txt
31edc83e-fd81-4c00-85f0-396701cbc0b5.mirbase21.mirnas.quantification.txt
31fa9bfd-dc91-4e27-be93-db7df8df2cd8.mirbase21.mirnas.quantification.txt
322dab3b-6516-45af-b25e-00524b73e861.mirbase21.mirnas.quantification.txt
331cccd2-f121-4ba8-80b9-b53f369b8add.mirbase21.mirnas.quantification.txt
3346e3c2-0458-4879-9522-d8c427b02b8c.mirbase21.mirn

4d2a183e-316b-43c3-b380-e1fcfe1577d1.mirbase21.mirnas.quantification.txt
4d63ecc3-6cba-4893-88bf-d7b927b99cd4.mirbase21.mirnas.quantification.txt
4d70b2a9-7ca6-4784-9e9b-15562a679821.mirbase21.mirnas.quantification.txt
4d89e455-8567-4e7e-9ddb-11fafb618735.mirbase21.mirnas.quantification.txt
4d9f4eb9-c576-431b-a7e3-745cc18dd39c.mirbase21.mirnas.quantification.txt
4dabe64a-7fb8-4009-a696-d4eee0cead8b.mirbase21.mirnas.quantification.txt
4db0f24e-8a4a-4249-8df8-97ca2b41770b.mirbase21.mirnas.quantification.txt
4edc18e9-dc60-4c91-a648-4e3e4652ef9b.mirbase21.mirnas.quantification.txt
4ee53bf1-9bfd-405c-a166-c88e665ab967.mirbase21.mirnas.quantification.txt
4f3dbf34-7147-4ae2-a874-f6755480a72a.mirbase21.mirnas.quantification.txt
4f569ea7-92d9-4a29-9f0e-8a5bfbdd717f.mirbase21.mirnas.quantification.txt
4f80ab99-6719-4626-92bc-c90a1b1ed724.mirbase21.mirnas.quantification.txt
4f9ab944-ee29-4f43-ac5d-61cee10483ac.mirbase21.mirnas.quantification.txt
4fbe4ee1-7efd-4ba4-8116-5ab3b4666b48.mirbase21.mirn

61d2f3b2-948b-45a0-b2f8-ed696e941c9b.mirbase21.mirnas.quantification.txt
61f63c03-128c-4c1d-a808-c9f41e521a1d.mirbase21.mirnas.quantification.txt
61fcb71c-2425-4514-85f7-7f1e63c9faab.mirbase21.mirnas.quantification.txt
62267940-4078-4506-a649-a8462c4dfee3.mirbase21.mirnas.quantification.txt
6295642d-d268-4485-be43-03405d5d2f26.mirbase21.mirnas.quantification.txt
6298c258-6d98-4c11-8a60-35cf63e7af86.mirbase21.mirnas.quantification.txt
62c3c334-cee3-4ac2-a904-9925497178a6.mirbase21.mirnas.quantification.txt
6330c24f-6984-44d4-84ff-5e4c34c806bc.mirbase21.mirnas.quantification.txt
634b29ab-b1da-48dd-8cf3-c15cbc03e9ff.mirbase21.mirnas.quantification.txt
636c9329-44fb-4dd4-b0d6-2bf0bc9531de.mirbase21.mirnas.quantification.txt
636e09a5-2922-48b9-8aab-875fbba5628d.mirbase21.mirnas.quantification.txt
6380cd41-2628-484f-acd3-2240d9903fba.mirbase21.mirnas.quantification.txt
640f2035-1d24-4f16-bd8d-0017fad45fa6.mirbase21.mirnas.quantification.txt
64302b1c-cfec-4d76-a67c-5455c68dbb89.mirbase21.mirn

7adaa246-6cbf-4be8-818a-75e3237dd669.mirbase21.mirnas.quantification.txt
7af20b1b-f239-4ebe-84b4-61f9c0abb402.mirbase21.mirnas.quantification.txt
7b1a8de2-9eda-44fc-bb91-8afd83e7bf0d.mirbase21.mirnas.quantification.txt
7b280634-96e6-4ad2-b636-ef9984e9d4c6.mirbase21.mirnas.quantification.txt
7b28237b-8d9e-4e89-bab9-e1c27739acba.mirbase21.mirnas.quantification.txt
7b3ccedc-9170-4e97-84aa-d6cb4c6d0727.mirbase21.mirnas.quantification.txt
7b4e72f6-b11a-4a47-bc20-2415f07e7792.mirbase21.mirnas.quantification.txt
7b6b8240-5620-45b0-bdf9-3e187cc240f4.mirbase21.mirnas.quantification.txt
7b79bd3a-561a-4aa6-9622-30ab20f63c61.mirbase21.mirnas.quantification.txt
7b7d84be-ef09-4af7-9c6d-845c936f8477.mirbase21.mirnas.quantification.txt
7b8b48de-1272-44cf-a753-422fa917a4e4.mirbase21.mirnas.quantification.txt
7bac4211-90ac-4564-95e2-4243fd34577a.mirbase21.mirnas.quantification.txt
7c163f61-6549-4561-9f29-5eab31e150ac.mirbase21.mirnas.quantification.txt
7c28a756-8d41-4077-9c6c-dfabe24078ed.mirbase21.mirn

907df606-c7be-483b-93b9-39fb4945d9c3.mirbase21.mirnas.quantification.txt
90e83adc-6d01-4959-b6bd-08d156f45b0f.mirbase21.mirnas.quantification.txt
9107ffc9-f542-4fc2-b896-b6c71c5d3a80.mirbase21.mirnas.quantification.txt
91151f2d-1b98-4671-8e79-05e07931e325.mirbase21.mirnas.quantification.txt
912bc7f5-fa03-4047-8edd-bd984a2a4d8d.mirbase21.mirnas.quantification.txt
919c05b9-7b6e-4a0e-9cbb-e25ef27984c7.mirbase21.mirnas.quantification.txt
91e0ab4f-ae82-43f4-855d-bb9ea23dd665.mirbase21.mirnas.quantification.txt
921bd908-7203-4250-bb94-7509e543cdb8.mirbase21.mirnas.quantification.txt
9293a93f-b3d1-41ab-8c18-3deed4cc776d.mirbase21.mirnas.quantification.txt
92d667b3-1714-46a7-8553-c390bef83d14.mirbase21.mirnas.quantification.txt
92da821d-45ac-4c29-b59b-98ed5ee15fc7.mirbase21.mirnas.quantification.txt
9310b2a3-d0eb-4139-baab-b4ed7bd0c61b.mirbase21.mirnas.quantification.txt
93430c10-3c95-4e41-9291-e0678fc8738c.mirbase21.mirnas.quantification.txt
93a009ac-6568-44e6-b575-6f3f44bfc4a2.mirbase21.mirn

a7956d39-fd8c-4072-8620-311a2de4844c.mirbase21.mirnas.quantification.txt
a81ef833-4483-43ad-b94c-7dbf75a2939b.mirbase21.mirnas.quantification.txt
a8c7bebe-a450-4a3e-b891-e2d4ac578b04.mirbase21.mirnas.quantification.txt
a94d1ab5-6c8a-4437-8c99-20a6c44cd09a.mirbase21.mirnas.quantification.txt
a956f9aa-d3f3-4130-8899-d7505e2a52e2.mirbase21.mirnas.quantification.txt
a98792e8-6f92-4d23-9ff5-40bf4b684cb2.mirbase21.mirnas.quantification.txt
a9b0cde1-d949-4dfa-80c7-a294cee61a2e.mirbase21.mirnas.quantification.txt
a9c72a75-2d8b-4f11-8af0-8cfed17e0ec4.mirbase21.mirnas.quantification.txt
aa4cdc70-f329-4065-a942-89596d9fef2a.mirbase21.mirnas.quantification.txt
aa75cc8a-33cd-4ab2-9532-57180d6bda4e.mirbase21.mirnas.quantification.txt
aa989d18-ba34-4085-87a6-3e56ba68c193.mirbase21.mirnas.quantification.txt
aaf189d3-d81f-4b4d-a0e1-5ed8df8d01ab.mirbase21.mirnas.quantification.txt
ab24cb18-c79f-48f4-8eec-6164ba4b5264.mirbase21.mirnas.quantification.txt
ab5523ce-da76-4318-b7f8-0a54345f8c29.mirbase21.mirn

bfd9eb35-ba0d-4c4c-b466-60bb5a197723.mirbase21.mirnas.quantification.txt
bfde45ed-90b9-4c38-bc38-d37291f983d2.mirbase21.mirnas.quantification.txt
bffe5d89-f332-49ac-bd92-8271144ed37d.mirbase21.mirnas.quantification.txt
c005b7df-5ff8-498f-a2ec-d89a46049517.mirbase21.mirnas.quantification.txt
c0405ebb-1311-40b2-a5c4-522d2cc62988.mirbase21.mirnas.quantification.txt
c074af13-a603-41f3-9537-375897680aaa.mirbase21.mirnas.quantification.txt
c0ecf541-285b-4344-b93b-0c7893d5fdcf.mirbase21.mirnas.quantification.txt
c1dc2366-2470-4854-8b20-08297c622e79.mirbase21.mirnas.quantification.txt
c231b1ae-ca2b-44c5-8ffe-b5db40b483b6.mirbase21.mirnas.quantification.txt
c2767263-4c29-45b0-84b4-1e351b52486b.mirbase21.mirnas.quantification.txt
c31d5759-bc16-4bbd-ad1d-5b4758d93158.mirbase21.mirnas.quantification.txt
c3356d61-896c-4bb7-a24a-7125b8bd57de.mirbase21.mirnas.quantification.txt
c3541530-dfa9-4f30-bbcc-6929e387d4ba.mirbase21.mirnas.quantification.txt
c357b7b6-593a-4450-b121-97f439ee58c2.mirbase21.mirn

d68be4db-0969-44e0-b0cc-1a0d7185c37a.mirbase21.mirnas.quantification.txt
d6a86f8b-ee8e-4e53-9590-b69a4ad551e5.mirbase21.mirnas.quantification.txt
d70152f4-d138-4d78-afab-ed6b14abe24e.mirbase21.mirnas.quantification.txt
d77e712b-0dcd-4922-9f83-b0bdc87675fc.mirbase21.mirnas.quantification.txt
d7b87911-9d07-4f76-b450-d7b0896b675c.mirbase21.mirnas.quantification.txt
d7c03f58-32b9-4dd4-9725-4c3880478dc3.mirbase21.mirnas.quantification.txt
d82c50f0-5b73-4e16-aae4-939d595a9471.mirbase21.mirnas.quantification.txt
d8812fe6-c5cc-49cd-a972-1963761b17ff.mirbase21.mirnas.quantification.txt
d88c989c-d8f6-427c-96b4-a5ebbe0bcbe5.mirbase21.mirnas.quantification.txt
d900f712-a98a-47fd-9a33-f13e57f5c343.mirbase21.mirnas.quantification.txt
d98eca57-7752-4f03-80a2-dac5c07957f0.mirbase21.mirnas.quantification.txt
d99e16ae-7815-4b80-ac00-ff1c28bb08ec.mirbase21.mirnas.quantification.txt
d9dc0777-5db7-4afa-bcfc-e047558e529b.mirbase21.mirnas.quantification.txt
da03b00f-2c56-47fe-8534-bf561863e209.mirbase21.mirn

ef57b156-a03d-4d91-a7d7-c88f9142e037.mirbase21.mirnas.quantification.txt
ef715b4a-c879-4528-856c-7a70e29b5819.mirbase21.mirnas.quantification.txt
efb16819-c795-4e94-b56d-ec857ee36902.mirbase21.mirnas.quantification.txt
efffe4d8-78f7-4acf-b27c-3daed222831a.mirbase21.mirnas.quantification.txt
f00dddc4-9a26-4569-aa6f-c6f65341283a.mirbase21.mirnas.quantification.txt
f01c1156-3363-43e1-9394-b1f88deba0d5.mirbase21.mirnas.quantification.txt
f04164db-982b-4709-a9d7-e3bd277b94d2.mirbase21.mirnas.quantification.txt
f06141f1-fb18-4bc2-b556-c4c64e35436f.mirbase21.mirnas.quantification.txt
f07ee060-4646-44b8-8271-b5a9bfc8876c.mirbase21.mirnas.quantification.txt
f090eb94-1318-46ef-b01a-23bb11eb3844.mirbase21.mirnas.quantification.txt
f0bfa597-6d1c-4ade-b296-621e4fec45a1.mirbase21.mirnas.quantification.txt
f122cf42-52e9-4fd1-8626-21aef93bdda9.mirbase21.mirnas.quantification.txt
f134347d-2168-4055-982f-ee572fa3704c.mirbase21.mirnas.quantification.txt
f162baf3-d93d-4c01-88d3-c298e95395c4.mirbase21.mirn

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from csv import reader
miRnaOutfileMatrix = './miRnaSeq/miRnaSeqFiles/outFileMatrix.csv'
# read csv file as a list of lists
with open(miRnaOutfile, 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Pass reader object to list() to get a list of lists
    miRnaOutfileMatrixCSV = list(csv_reader)
    #print(miRnaOutfileMatrixCSV)
    
type(miRnaOutfileMatrixCSV)
print(miRnaOutfileMatrixCSV[900])

In [ ]:
miRnaOutfileMatrix = './miRnaSeq/miRnaSeqFiles/outFileMatrix.csv'
outf = './miRnaSeq/miRnaSeqFiles/dummyOut.csv'
miRnaOutfileMatrixCSV = downloadMiRnaData(miRnaOutfileMatrix, case_sensitive=True)

#type(miRnaOutfileMatrixCSV)
miRnaOutfileMatrixCSV = pd.DataFrame(miRnaOutfileMatrixCSV).to_numpy()
type(miRnaOutfileMatrixCSV)
#arr.dtype()
#np.array(miRnaOutfileMatrixCSV.tolist())
#print(type(miRnaOutfileMatrixCSV))

#csv = writeToCsv(outf, miRnaOutfileMatrix)

In [ ]:
clinical = './clinical.tsv'
miRnaOutfile = './miRnaSeq/miRnaSeqFiles/outFile.csv'

fullMatrix = addYforPatient(miRnaOutfileMatrixCSV, clinical, miRnaOutfile)

print(fullMatrix)

findPat TCGA-BH-A0H3 Stage I
476
findPat TCGA-A8-A09T '--
477
findPat TCGA-EW-A2FV Stage IIIC
478
findPat TCGA-A7-A13G Stage IIA
479
findPat TCGA-AR-A24V Stage IIB
480
findPat TCGA-AO-A03T Stage IIB
481
findPat TCGA-BH-A0AZ Stage IIIA
482
findPat TCGA-BH-A0DO Stage I
483
findPat TCGA-BH-A1FJ Stage IIIA
484
findPat TCGA-D8-A140 Stage IIB
485
findPat TCGA-BH-A0C0 Stage IIA
486
findPat TCGA-A2-A0D4 Stage IIB
487
findPat TCGA-PE-A5DC Stage IIIA
488
findPat TCGA-C8-A8HR Stage IIIA
489
findPat TCGA-D8-A3Z5 Stage IIIC
490
findPat TCGA-BH-A1FD Stage I
491
findPat TCGA-A8-A09I Stage IIA
492
findPat TCGA-BH-A0BQ Stage I
493
findPat TCGA-BH-A0BO Stage I
494
findPat TCGA-BH-A18P Stage I
495
findPat TCGA-AO-A12E Stage IIB
496
findPat TCGA-OL-A5DA Stage IIA
497
findPat TCGA-BH-A42U Stage IIA
498
findPat TCGA-B6-A0IG Stage IIB
499
findPat TCGA-WT-AB44 Stage IA
500
findPat TCGA-AO-A126 Stage IIA
501
findPat TCGA-GM-A2DD Stage I
502
findPat TCGA-A2-A25A Stage IIA
503
findPat TCGA-AN-A04A Stage IIIA
504

findPat TCGA-C8-A133 Stage IIIA
712
findPat TCGA-C8-A1HI Stage IIIA
713
findPat TCGA-BH-A0DX Stage I
714
findPat TCGA-AR-A24W Stage IIB
715
findPat TCGA-BH-AB28 Stage IIIA
716
findPat TCGA-AR-A24U Stage IIA
717
findPat TCGA-C8-A12Q Stage IIIA
718
findPat TCGA-BH-A0BA Stage IIIC
719
findPat TCGA-D8-A1X5 Stage IIIC
720
findPat TCGA-A8-A082 Stage IIB
721
findPat TCGA-AR-A251 Stage IIIA
722
findPat TCGA-AR-A0TX Stage IIA
723
findPat TCGA-A8-A097 Stage IIB
724
findPat TCGA-GM-A3NY Stage IIB
725
findPat TCGA-B6-A0IB Stage IV
726
findPat TCGA-BH-A204 Stage IIB
727
findPat TCGA-BH-A0H9 Stage IIA
728
findPat TCGA-AN-A0AS Stage IIIA
729
findPat TCGA-UU-A93S Stage IV
730
findPat TCGA-A8-A08A Stage I
731
findPat TCGA-E9-A24A Stage IIA
732
findPat TCGA-B6-A0RU Stage IA
733
findPat TCGA-E9-A1RI Stage IIIA
734
findPat TCGA-AC-A3OD Stage IIB
735
findPat TCGA-BH-A1FN Stage IIA
736
findPat TCGA-A7-A6VW Stage IIA
737
findPat TCGA-D8-A1Y0 Stage IIIA
738
findPat TCGA-AQ-A1H2 Stage IIIA
739
findPat TCGA-D8-

findPat TCGA-BH-A0DV Stage IIIA
947
findPat TCGA-OL-A66O Stage IIB
948
findPat TCGA-AC-A6IW Stage IIA
949
findPat TCGA-E2-A1BC Stage IA
950
findPat TCGA-AO-A12C Stage IIB
951
findPat TCGA-B6-A0X0 Stage I
952
findPat TCGA-AO-A03O Stage IIA
953
findPat TCGA-A7-A0DC Stage IA
954
findPat TCGA-E2-A576 Stage IA
955
findPat TCGA-A8-A09M Stage IIIC
956
findPat TCGA-BH-A2L8 Stage IIA
957
findPat TCGA-AC-A5EI Stage IB
958
findPat TCGA-BH-A1EO Stage IIA
959
findPat TCGA-C8-A26X Stage IIA
960
findPat TCGA-AR-A250 Stage IIA
961
findPat TCGA-EW-A1OZ Stage IIA
962
findPat TCGA-B6-A0RT Stage IIIA
963
findPat TCGA-AR-A2LR Stage I
964
findPat TCGA-B6-A0X7 Stage X
965
findPat TCGA-LL-A7T0 Stage IIB
966
findPat TCGA-A8-A08O Stage IV
967
findPat TCGA-BH-A0AY Stage IIA
968
findPat TCGA-D8-A1Y3 Stage IIIA
969
findPat TCGA-C8-A12O Stage IIA
970
findPat TCGA-E9-A5FL Stage IIB
971
findPat TCGA-BH-A0BT Stage IIA
972
findPat TCGA-OL-A5RU Stage IIA
973
findPat TCGA-E9-A3X8 Stage IIB
974
findPat TCGA-B6-A409 Stage 

findPat TCGA-D8-A27G Stage IIA
1177
findPat TCGA-AC-A3HN Stage IIB
1178
findPat TCGA-E2-A15P Stage IA
1179
findPat TCGA-E2-A14Y Stage IIA
1180
findPat TCGA-BH-A1EO Stage IIA
1181
findPat TCGA-B6-A1KF Stage IIB
1182
findPat TCGA-BH-A1EU Stage I
1183
findPat TCGA-D8-A1XR Stage IIB
1184
findPat TCGA-BH-A1FE Stage IIB
1185
findPat TCGA-Z7-A8R6 Stage I
1186
findPat TCGA-PL-A8LY Stage IIB
1187
findPat TCGA-E2-A574 Stage IIA
1188
findPat TCGA-E2-A153 Stage IIB
1189
findPat TCGA-E9-A1RH Stage IIA
1190
findPat TCGA-B6-A0RV Stage IIIA
1191
findPat TCGA-AN-A0AL Stage IIIB
1192
findPat TCGA-BH-A18F Stage IIA
1193
findPat TCGA-E9-A1R4 Stage IA
1194
findPat TCGA-A2-A4RW Stage IIIC
1195
findPat TCGA-LL-A440 Stage IA
1196
findPat TCGA-BH-A8FZ Stage IA
1197
findPat TCGA-A8-A09B Stage IIIB
1198
findPat TCGA-A2-A0YK Stage IIA
1199
findPat TCGA-BH-A202 Stage II
1200
findPat TCGA-BH-A1F0 Stage IIA
1201
findPat TCGA-LL-A5YL Stage IIB
1202
findPat TCGA-A2-A3XV Stage IIA
1203
TCGA-E2-A14S TCGA-BH-A0B2
findPat

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# File lengths (features)

In [ ]:
miRnaOutfile = './miRnaSeq/miRnaSeqFiles/outFile.csv'
rnaOutfile = './rnaSeq/outFile.csv'

from csv import reader

with open(miRnaOutfile, 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Pass reader object to list() to get a list of lists
    miRnaOutfileMatrixCSV = list(csv_reader)
    #print(miRnaOutfileMatrixCSV)
    
with open(rnaOutfile, 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Pass reader object to list() to get a list of lists
    rnaOutfileMatrixCSV = list(csv_reader)
    #print(miRnaOutfileMatrixCSV)
    
len(miRnaOutfileMatrixCSV[0])
len(rnaOutfileMatrixCSV[0])

print('miRna', len(miRnaOutfileMatrixCSV[0]))
print('rna', len(rnaOutfileMatrixCSV[0]))
#type(miRnaOutfileMatrixCSV)


# Methylation

In [ ]:
from gtfparse import read_gtf
import pandas as pd
import tqdm
import os
import numpy as np
import warnings
import gc

warnings.filterwarnings('ignore')

def downloadMathylData(fname, **kwargs):
    def rewrite_csv_as_tab(fname):
        with open(fname, newline='') as fp:
            dialect = csv.Sniffer().sniff(fp.readline())
            fp.seek(0)
            for row in csv.reader(fp, dialect):
                yield "\t".join(row)
    return np.recfromcsv(rewrite_csv_as_tab(fname), delimiter='\t', encoding=None, **kwargs)

def get_all_gene_id():
    path = './methylation/methylationFiles/files/00a82e51-7dd9-4e30-92ab-39d900b9a1ba.methylation_array.sesame.level3betas.txt'
    return pd.read_csv(path, index_col=0).columns.values


def get_gene_intervals(gene_data):
    #g = gene_data.groupby(by='strand')
    """
    g = gene_data.groupby(by='strand')
    gene_data['start'][g.get_group('+').index]['start'] = gene_data.loc[g.get_group('+').index]['start'].apply(
        lambda x: x - 1000)
    gene_data['end'][g.get_group('-').index] = gene_data.loc[g.get_group('-').index]['end'].apply(
        lambda x: x + 1000)"""
    
    gene_data['start'] = gene_data['start'].apply(
        lambda x: x - 1000)
    gene_data['end'] = gene_data['end'].apply(
        lambda x: x + 1000)
    # gene_data.drop(columns=['strand'], inplace=True)
    # gene_data.sort_values(by=['start', 'end'], inplace=True)
    gene_id = gene_data['gene_id'].unique()
    gene_data_new = pd.DataFrame(np.zeros_like(gene_id).reshape(-1, 1), columns=['Intervals'], index=gene_id)
    g = gene_data.groupby(by='gene_id')
    for id in tqdm.tqdm(gene_id):
        tmp = g.get_group(id)
        # gene_data_new.loc[id] = pd.arrays.IntervalArray.from_arrays(left=tmp['start'].values,
        #                                                             right=tmp['end'].values, closed='both')
        gene_data_new['Intervals'][id] = pd.arrays.IntervalArray.from_arrays(left=tmp['start'].values,
                                                                             right=tmp['end'].values, closed='both')
    return gene_data_new


def create_gene_expression_sheet(RNA_sheet, column='fpkm_uq_unstranded', is_save=True):
    gene_id = get_all_gene_id()
    df = pd.DataFrame(columns=gene_id)
    for index, row in tqdm.tqdm(RNA_sheet.iterrows()):
        case = pd.read_table('data/RNA-Sqe & Methylation/' + row['File ID'] + '/' + row['File Name'], header=1).iloc[4:]
        df.loc[row['Case ID']] = case[column].values
    if is_save:
        df.to_csv('data/gene_expression_sheet.csv')
    return df


def create_rna_methylation_sheet(methylation_sheet, gene_data, directory, is_save=True):
    gene_id = get_all_gene_id()
    df = pd.DataFrame(columns=gene_id)
    gene_id = np.intersect1d(gene_data.index.unique().values, gene_id)
    gene_data = gene_data.loc[gene_id]
    for root,dirs,files in os.walk(directory):
        files = [f for f in files if not f[0] == '.']
        dirs[:] = [d for d in dirs if not d[0] == '.']
        for file in files:
            #print(file)
            if file.endswith(".txt"):
                path = directory + '/' + file
                meth_data = pd.read_csv(path, sep="\t", header=None)
                tmp = meth_data[0].apply(lambda x: x[:2])
                cg_pairs_index = tmp[tmp == 'cg'].index
                meth_data = meth_data.loc[cg_pairs_index]
                location = meth_data[0].apply(lambda x: int(x[2:]))
                meth_data.set_index(0, inplace=True)
                map_path = './methylation/methylationFiles/temp/' + file[:-4] + '_rna_methylation_map.csv'
                os.makedirs('./methylation/methylationFiles/temp/', exist_ok=True)  
                if os.path.exists(map_path):
                    rna_methylation_map = pd.read_csv(map_path, index_col=0)
                    func = lambda x: x[1: -1].replace('\n', '').replace('... ', '').replace("'", '').split(' ') if x != '['']' else []
                    for id in gene_id:
                        rna_methylation_map.loc[id, '0'] = func(rna_methylation_map.loc[id, '0'])
                else:
                    rna_methylation_map = pd.DataFrame(index=gene_id, columns=['Indicator'])
                    print('Generating the RNA-Methylation Map...')
                    for id in tqdm.tqdm(gene_id):
                        is_methylated = location.apply(lambda x: np.any(gene_data.loc[id, 'Intervals'].contains(x))).values
                        rna_methylation_map.loc[id, 'Indicator'] = meth_data[is_methylated].index.values
                    rna_methylation_map.to_csv(map_path)
                    
                for index, row in tqdm.tqdm(methylation_sheet.iterrows()):
                    #file = './methylation/methylationFiles/files/' + row['File Name']
                    print(file)
                    meth_data = downloadMathylData('./methylation/methylationFiles/files/' + file)
                    #meth_data = pd.read_table('./methylation/methylationFiles/files/' + row['File Name'], header=None)
                    meth_data = pd.DataFrame(meth_data)
                    #meth_data.set_index(0, inplace=True)
                    #meth_data.fillna(value=0, inplace=True)
                    if len(meth_data) != 485577:
                        continue
                    func = lambda x: np.mean(meth_data.loc[rna_methylation_map.loc[x, '0']].values)
                    df.loc[row['Case ID'], gene_id] = np.vectorize(func)(gene_id)
                df.fillna(value=0, inplace=True)
                print("done tmp")
                
                if is_save:
                    df.to_csv('./methylation/methylationFiles/outfile.csv')
        return df
                
                
        #meth_data = pd.read('./methylation/methylationFiles/files/'+ methylation_sheet['File Name'].iloc[0], header=None)
        #tmp = meth_data[0].apply(lambda x: x[:2])
        #cg_pairs_index = tmp[tmp == 'cg'].index
        #meth_data = meth_data.loc[cg_pairs_index]
        #location = meth_data[0].apply(lambda x: int(x[2:]))
        #meth_data.set_index(0, inplace=True)
        #map_path = 'data/rna_methylation_map.csv'
        """if os.path.exists(map_path):
            rna_methylation_map = pd.read_csv(map_path, index_col=0)
            func = lambda x: x[1: -1].replace('\n', '').replace('... ', '').replace("'", '').split(' ') if x != '['']' else []
            for id in gene_id:
                rna_methylation_map.loc[id, '0'] = func(rna_methylation_map.loc[id, '0'])
        else:
            rna_methylation_map = pd.DataFrame(index=gene_id, columns=['Indicator'])
            print('Generating the RNA-Methylation Map...')
            for id in tqdm.tqdm(gene_id):
                is_methylated = location.apply(lambda x: np.any(gene_data.loc[id, 'Intervals'].contains(x))).values
                rna_methylation_map.loc[id, 'Indicator'] = meth_data[is_methylated].index.values
            rna_methylation_map.to_csv(map_path)"""

        """
        for index, row in tqdm.tqdm(methylation_sheet.iterrows()):
            meth_data = pd.read_table('./methylation/methylationFiles/files/' + row['File ID'] + '/' + row['File Name'], header=None)
            meth_data.set_index(0, inplace=True)
            meth_data.fillna(value=0, inplace=True)
            if len(meth_data) != 485577:
                continue
            func = lambda x: np.mean(meth_data.loc[rna_methylation_map.loc[x, '0']].values)
            df.loc[row['Case ID'], gene_id] = np.vectorize(func)(gene_id)
        df.fillna(value=0, inplace=True)"""
        
        """if is_save:
            df.to_csv('data/rna_methylation_sheet.csv')
        return df"""



In [ ]:
os.getcwd()
os.chdir("../../..")
os.getcwd()
os.listdir()
os.chdir("/Volumes/Toshiba_blue")
os.listdir()
os.getcwd()

In [ ]:
# gene_data = read_gtf('data/gencode.v41.chr_patch_hapl_scaff.annotation.gtf')
# gene_data = gene_data[['gene_id', 'start', 'end', 'strand']]
gene_data = pd.read_csv('./methylation/genecode1.csv')
gene_data = get_gene_intervals(gene_data)
print("done")


In [ ]:
print(gene_data)

In [ ]:
sample_sheet = pd.read_table('./methylation/methylationSampleSheet.tsv')
methylation_sheet = sample_sheet[sample_sheet['Data Category'] == 'DNA Methylation'][['File ID', 'File Name', 'Case ID']]
methylation_sheet.drop_duplicates(subset='Case ID', inplace=True)


In [ ]:
df = create_rna_methylation_sheet(methylation_sheet, gene_data, "./methylation/methylationFiles/files")

In [ ]:
print(df)